In [1]:
import numpy as np
import pandas as pd
import autokeras as ak
import os

from scipy.stats import spearmanr
from sklearn.decomposition import PCA

In [2]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
PATH = '../DrugCell/data_rcellminer/'

t = pd.read_table(PATH + 'train_rcell_wo_other.txt', header=None)
t.columns = ['CELL_LINE', 'SMILES', 'label']
t = reduce_mem_usage(t)

pca = PCA(n_components=300)

mfp = pd.read_csv(PATH + 'drug2fingerprint.csv', header=None,)
mfp = pd.DataFrame(pca.fit_transform(mfp))
mfp['SMILES'] = list(pd.read_csv(PATH + 'drug2ind.txt', header=None, sep='\t')[1])
mfp = reduce_mem_usage(mfp)

pca = PCA(n_components=50)

cell_line = pd.read_csv(PATH + 'cell2mut.txt', header=None)
cell_line = pd.DataFrame(pca.fit_transform(cell_line))
cell_line['CELL_LINE'] = pd.read_csv(PATH + 'cell2ind.txt', sep='\t', header=None)[1]
cell_line = reduce_mem_usage(cell_line)

t = t.merge(mfp, on='SMILES')    
train = t.merge(cell_line, on='CELL_LINE')

Memory usage of dataframe is 0.23 MB
Memory usage after optimization is: 0.06 MB
Decreased by 73.5%
Memory usage of dataframe is 0.71 MB
Memory usage after optimization is: 0.19 MB
Decreased by 73.5%
Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.01 MB
Decreased by 63.5%


In [4]:
mfp.shape

(309, 301)

In [5]:
cell_line.shape

(55, 51)

In [6]:
reg = ak.StructuredDataRegressor(
    overwrite=True, 
    max_trials=100
) 

In [7]:
t = pd.read_table(
    PATH + 'val_rcell_wo_other.txt', 
    header=None
    )
t.columns = ['CELL_LINE', 'SMILES', 'label']
t = reduce_mem_usage(t)
t = t.merge(mfp, on='SMILES')    
t = t.merge(cell_line, on='CELL_LINE')

Memory usage of dataframe is 0.08 MB
Memory usage after optimization is: 0.03 MB
Decreased by 62.1%


In [8]:
reg.fit(
    train.drop('label', axis=1),
    train['label'],
    validation_data=(t.drop('label', axis=1), t['label']),
    epochs=300,
)

Trial 60 Complete [00h 02m 15s]
val_loss: 0.8749184608459473

Best val_loss So Far: 0.8585431575775146
Total elapsed time: 02h 43m 01s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/300
308/308 [==============================] - 6s 9ms/step - loss: 1.0096 - mean_squared_error: 1.0096 - val_loss: 0.8886 - val_mean_squared_error: 0.8886
Epoch 2/300
308/308 [==============================] - 1s 4ms/step - loss: 0.9210 - mean_squared_error: 0.9210 - val_loss: 0.8866 - val_mean_squared_error: 0.8866
Epoch 3/300
308/308 [==============================] - 1s 4ms/step - loss: 0.9122 - mean_squared_error: 0.9122 - val_loss: 0.8859 - val_mean_squared_error: 0.8859
Epoch 4/300
308/308 [==============================] - 1s 4ms/step - loss: 0.9086 - mean_squared_error: 0.9086 - val_loss: 0.8874 - val_mean_squared_error: 0.8874
Epoch 5/300
308/308 [==============================] - 1s 4ms/step - loss: 0.9010 - mean_squared_error: 0.9010 - val_loss: 0.8859 - val_mean_squared_error: 0.8859
Epoch 6/300
308/308 [==============================] - 1s 4ms/step - loss: 0.9033 - mean_squared_error: 0.9033 - val_loss: 0.8859 - val_mean_squared_error: 0.8859
Epoch 7/300
308/308 [=

308/308 [==============================] - 1s 4ms/step - loss: 0.8746 - mean_squared_error: 0.8746 - val_loss: 0.8746 - val_mean_squared_error: 0.8746
Epoch 52/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8697 - mean_squared_error: 0.8697 - val_loss: 0.8740 - val_mean_squared_error: 0.8740
Epoch 53/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8720 - mean_squared_error: 0.8720 - val_loss: 0.8741 - val_mean_squared_error: 0.8741
Epoch 54/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8720 - mean_squared_error: 0.8720 - val_loss: 0.8749 - val_mean_squared_error: 0.8749
Epoch 55/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8687 - mean_squared_error: 0.8687 - val_loss: 0.8741 - val_mean_squared_error: 0.8741
Epoch 56/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8704 - mean_squared_error: 0.8704 - val_loss: 0.8738 - val_mean_squared_error: 0.8738
Epoch 57/300
308/308 [=======

308/308 [==============================] - 1s 4ms/step - loss: 0.8500 - mean_squared_error: 0.8500 - val_loss: 0.8713 - val_mean_squared_error: 0.8713
Epoch 102/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8537 - mean_squared_error: 0.8537 - val_loss: 0.8713 - val_mean_squared_error: 0.8713
Epoch 103/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8535 - mean_squared_error: 0.8535 - val_loss: 0.8705 - val_mean_squared_error: 0.8705
Epoch 104/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8491 - mean_squared_error: 0.8491 - val_loss: 0.8711 - val_mean_squared_error: 0.8711
Epoch 105/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8513 - mean_squared_error: 0.8513 - val_loss: 0.8702 - val_mean_squared_error: 0.8702
Epoch 106/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8513 - mean_squared_error: 0.8513 - val_loss: 0.8716 - val_mean_squared_error: 0.8716
Epoch 107/300
308/308 [=

Epoch 151/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8307 - mean_squared_error: 0.8307 - val_loss: 0.8680 - val_mean_squared_error: 0.8680
Epoch 152/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8322 - mean_squared_error: 0.8322 - val_loss: 0.8661 - val_mean_squared_error: 0.8661
Epoch 153/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8318 - mean_squared_error: 0.8318 - val_loss: 0.8660 - val_mean_squared_error: 0.8660
Epoch 154/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8311 - mean_squared_error: 0.8311 - val_loss: 0.8659 - val_mean_squared_error: 0.8659
Epoch 155/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8303 - mean_squared_error: 0.8303 - val_loss: 0.8670 - val_mean_squared_error: 0.8670
Epoch 156/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8281 - mean_squared_error: 0.8281 - val_loss: 0.8658 - val_mean_squared_error: 0.8658
Epoch 157/

308/308 [==============================] - 1s 4ms/step - loss: 0.8093 - mean_squared_error: 0.8093 - val_loss: 0.8618 - val_mean_squared_error: 0.8618
Epoch 201/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8100 - mean_squared_error: 0.8100 - val_loss: 0.8612 - val_mean_squared_error: 0.8612
Epoch 202/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8082 - mean_squared_error: 0.8082 - val_loss: 0.8612 - val_mean_squared_error: 0.8612
Epoch 203/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8071 - mean_squared_error: 0.8071 - val_loss: 0.8614 - val_mean_squared_error: 0.8614
Epoch 204/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8108 - mean_squared_error: 0.8108 - val_loss: 0.8621 - val_mean_squared_error: 0.8621
Epoch 205/300
308/308 [==============================] - 1s 4ms/step - loss: 0.8055 - mean_squared_error: 0.8055 - val_loss: 0.8616 - val_mean_squared_error: 0.8616
Epoch 206/300
308/308 [=

Epoch 250/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7895 - mean_squared_error: 0.7895 - val_loss: 0.8585 - val_mean_squared_error: 0.8585
Epoch 251/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7862 - mean_squared_error: 0.7862 - val_loss: 0.8600 - val_mean_squared_error: 0.8600
Epoch 252/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7872 - mean_squared_error: 0.7872 - val_loss: 0.8582 - val_mean_squared_error: 0.8582
Epoch 253/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7852 - mean_squared_error: 0.7852 - val_loss: 0.8592 - val_mean_squared_error: 0.8592
Epoch 254/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7845 - mean_squared_error: 0.7845 - val_loss: 0.8586 - val_mean_squared_error: 0.8586
Epoch 255/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7896 - mean_squared_error: 0.7896 - val_loss: 0.8601 - val_mean_squared_error: 0.8601
Epoch 256/

308/308 [==============================] - 1s 4ms/step - loss: 0.7698 - mean_squared_error: 0.7698 - val_loss: 0.8631 - val_mean_squared_error: 0.8631
Epoch 300/300
308/308 [==============================] - 1s 4ms/step - loss: 0.7734 - mean_squared_error: 0.7734 - val_loss: 0.8613 - val_mean_squared_error: 0.8613
INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [9]:
t = pd.read_table(
    PATH + 'test_rcell_wo_other.txt', 
    header=None
    )
t.columns = ['CELL_LINE', 'SMILES', 'label']
t = reduce_mem_usage(t)
t = t.merge(mfp, on='SMILES')    
t = t.merge(cell_line, on='CELL_LINE')

Memory usage of dataframe is 0.08 MB
Memory usage after optimization is: 0.03 MB
Decreased by 62.1%


In [10]:
predicted_y = reg.predict(t.drop('label', axis=1))

103/103 [==============================] - 0s 4ms/step


In [11]:
spearmanr(pd.DataFrame(predicted_y)[0], t['label'])

SpearmanrResult(correlation=0.19351239352428096, pvalue=5.253350993299896e-29)